In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
! pip install alpha-vantage
from alpha_vantage.timeseries import TimeSeries

In [ ]:
demo = 'NV2Y5IAVZKQOFHKX' #YOUR API KEY HERE

In [ ]:
url_global_deaths = r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
url_global_confirmed = r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url_retail_stock = r'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=NYSE:SPG&apikey=' + demo + '&datatype=csv'
url_usa_stock = r'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=' + demo + '&datatype=csv'
url_canada_stock = r'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=TSE:BB&apikey=' + demo + '&datatype=csv'
url_travel_stock = r'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=EXPE&apikey=' + demo + '&datatype=csv'
url_material_stock = r'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=NYSE:GOLD&apikey=' + demo + '&datatype=csv'

In [ ]:
deaths_df = pd.read_csv(url_global_deaths)
confirm_df = pd.read_csv(url_global_confirmed)
us_stock = pd.read_csv(url_usa_stock)
ca_stock = pd.read_csv(url_canada_stock)
retail_stock = pd.read_csv(url_retail_stock)
travel_stock = pd.read_csv(url_travel_stock)
material_stock = pd.read_csv(url_material_stock)

In [ ]:
print(deaths_df.shape)
print(confirm_df.shape)
print(us_stock.shape)
print(ca_stock.shape)
print(retail_stock.shape)
print(travel_stock.shape)
print(material_stock.shape)

In [ ]:
deaths_df_long = pd.melt(deaths_df, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                        value_vars= deaths_df.columns[4:], var_name='timestamp', value_name='death_cases')

confirm_df_long = pd.melt(confirm_df, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                        value_vars= confirm_df.columns[4:], var_name='timestamp', value_name='confirmed_cases')

In [ ]:
deaths_df_long.head()

In [ ]:
confirm_df_long.head()

In [ ]:
us_stock['stock_name'] = 'MSFT'
ca_stock['stock_name'] = 'BB'
retail_stock['stock_name'] = 'SPG'
travel_stock['stock_name'] = 'EXPE'
material_stock['stock_name'] = 'GOLD'

In [ ]:
final_stock = pd.concat([us_stock, ca_stock, retail_stock, travel_stock, material_stock]).reset_index(drop = True)
final_stock.stock_name.unique()

In [ ]:
final_stock.shape

In [ ]:
final_stock.tail()

In [ ]:
deaths_df_long = deaths_df_long.groupby(['Country/Region','timestamp'])['death_cases'].sum().reset_index()
confirm_df_long = confirm_df_long.groupby(['Country/Region','timestamp'])['confirmed_cases'].sum().reset_index()
final_covid = pd.concat([deaths_df_long, confirm_df_long['confirmed_cases']], axis = 1)

In [ ]:
final_covid.head()

In [ ]:
final_covid.tail()

In [ ]:
final_covid['timestamp'] = pd.to_datetime(final_covid.timestamp)
final_stock['timestamp'] = pd.to_datetime(final_stock.timestamp)

In [ ]:
final_data = final_stock.merge(final_covid, on = 'timestamp', how = 'left')

In [ ]:
final_data.head()

In [ ]:
final_data.shape

In [ ]:
final_data.timestamp.min()

In [ ]:
final_data.timestamp.max()

In [ ]:
final_data.isna().sum()

In [ ]:
MSFT_final = final_data[final_data.stock_name == 'MSFT']

In [ ]:
MSFT_final.head()

In [ ]:
MSFT_final1 = pd.DataFrame()
MSFT_final1['timestamp'] = MSFT_final[MSFT_final['Country/Region'] == 'US'].timestamp
MSFT_final1['death_cases'] = (MSFT_final[MSFT_final['Country/Region'] == 'US'].death_cases - MSFT_final[MSFT_final['Country/Region'] == 'US'].death_cases.mean())/MSFT_final[MSFT_final['Country/Region'] == 'US'].death_cases.std()
MSFT_final1['confirmed_cases'] = (MSFT_final[MSFT_final['Country/Region'] == 'US'].confirmed_cases - MSFT_final[MSFT_final['Country/Region'] == 'US'].confirmed_cases.mean())/MSFT_final[MSFT_final['Country/Region'] == 'US'].confirmed_cases.std()
MSFT_final1['high'] = (MSFT_final[MSFT_final['Country/Region'] == 'US'].high - MSFT_final[MSFT_final['Country/Region'] == 'US'].high.mean())/MSFT_final[MSFT_final['Country/Region'] == 'US'].high.std()
MSFT_final1['low'] = (MSFT_final[MSFT_final['Country/Region'] == 'US'].low - MSFT_final[MSFT_final['Country/Region'] == 'US'].low.mean())/MSFT_final[MSFT_final['Country/Region'] == 'US'].low.std()
MSFT_final1['open'] = (MSFT_final[MSFT_final['Country/Region'] == 'US'].open - MSFT_final[MSFT_final['Country/Region'] == 'US'].open.mean())/MSFT_final[MSFT_final['Country/Region'] == 'US'].open.std()
MSFT_final1['close'] = (MSFT_final[MSFT_final['Country/Region'] == 'US'].close - MSFT_final[MSFT_final['Country/Region'] == 'US'].close.mean())/MSFT_final[MSFT_final['Country/Region'] == 'US'].close.std()

In [ ]:
MSFT_final1.head()

In [ ]:
plt.plot(MSFT_final1.timestamp, MSFT_final1.death_cases, label = 'death_cases')
plt.plot(MSFT_final1.timestamp, MSFT_final1.confirmed_cases, label = 'confirmed_cases')
plt.plot(MSFT_final1.timestamp, MSFT_final1.high, label = 'high')
plt.plot(MSFT_final1.timestamp, MSFT_final1.low, label = 'low')
plt.plot(MSFT_final1.timestamp, MSFT_final1.open, label = 'open')
plt.plot(MSFT_final1.timestamp, MSFT_final1.close, label = 'close')
plt.legend()
plt.title('Stock Analysis Microsoft USA')
plt.show()

In [ ]:
BB_final = final_data[final_data.stock_name == 'BB']
BB_final1 = pd.DataFrame()
BB_final1['timestamp'] = BB_final[BB_final['Country/Region'] == 'Canada'].timestamp
BB_final1['death_cases'] = (BB_final[BB_final['Country/Region'] == 'Canada'].death_cases - BB_final[BB_final['Country/Region'] == 'Canada'].death_cases.mean())/BB_final[BB_final['Country/Region'] == 'Canada'].death_cases.std()
BB_final1['confirmed_cases'] = (BB_final[BB_final['Country/Region'] == 'Canada'].confirmed_cases - BB_final[BB_final['Country/Region'] == 'Canada'].confirmed_cases.mean())/BB_final[BB_final['Country/Region'] == 'Canada'].confirmed_cases.std()
BB_final1['high'] = (BB_final[BB_final['Country/Region'] == 'Canada'].high - BB_final[BB_final['Country/Region'] == 'Canada'].high.mean())/BB_final[BB_final['Country/Region'] == 'Canada'].high.std()
BB_final1['low'] = (BB_final[BB_final['Country/Region'] == 'Canada'].low - BB_final[BB_final['Country/Region'] == 'Canada'].low.mean())/BB_final[BB_final['Country/Region'] == 'Canada'].low.std()
BB_final1['open'] = (BB_final[BB_final['Country/Region'] == 'Canada'].open - BB_final[BB_final['Country/Region'] == 'Canada'].open.mean())/BB_final[BB_final['Country/Region'] == 'Canada'].open.std()
BB_final1['close'] = (BB_final[BB_final['Country/Region'] == 'Canada'].close - BB_final[BB_final['Country/Region'] == 'Canada'].close.mean())/BB_final[BB_final['Country/Region'] == 'Canada'].close.std()

In [ ]:
plt.plot(BB_final1.timestamp, BB_final1.death_cases, label = 'death_cases')
plt.plot(BB_final1.timestamp, BB_final1.confirmed_cases, label = 'confirmed_cases')
plt.plot(BB_final1.timestamp, BB_final1.high, label = 'high')
plt.plot(BB_final1.timestamp, BB_final1.low, label = 'low')
plt.plot(BB_final1.timestamp, BB_final1.open, label = 'open')
plt.plot(BB_final1.timestamp, BB_final1.close, label = 'close')
plt.legend()
plt.title('Stock Analysis Blackberry Canada')
plt.show()


In [ ]:
SPG_final = final_data[final_data.stock_name == 'SPG']

In [ ]:
SPG_final.shape

In [ ]:
SPG_final.head()

In [ ]:
SPG_final1 = SPG_final.groupby('timestamp')['open','high','low','close'].mean().reset_index().merge(SPG_final.groupby('timestamp')['death_cases','confirmed_cases'].sum().reset_index(),
                                                                                                   on = 'timestamp')

In [ ]:
SPG_final1.shape

In [ ]:
SPG_final1['death_cases'] = (SPG_final1.death_cases - SPG_final1.death_cases.mean())/SPG_final1.death_cases.std()
SPG_final1['confirmed_cases'] = (SPG_final1.confirmed_cases - SPG_final1.confirmed_cases.mean())/SPG_final1.confirmed_cases.std()
SPG_final1['high'] = (SPG_final1.high - SPG_final1.high.mean())/SPG_final1.high.std()
SPG_final1['low'] = (SPG_final1.low - SPG_final1.low.mean())/SPG_final1.low.std()
SPG_final1['open'] = (SPG_final1.open - SPG_final1.open.mean())/SPG_final1.open.std()
SPG_final1['close'] = (SPG_final1.close - SPG_final1.close.mean())/SPG_final1.close.std()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(SPG_final1.timestamp, SPG_final1.death_cases, label = 'death_cases')
plt.plot(SPG_final1.timestamp, SPG_final1.confirmed_cases, label = 'confirmed_cases')
plt.plot(SPG_final1.timestamp, SPG_final1.high, label = 'high')
plt.plot(SPG_final1.timestamp, SPG_final1.low, label = 'low')
plt.plot(SPG_final1.timestamp, SPG_final1.open, label = 'open')
plt.plot(SPG_final1.timestamp, SPG_final1.close, label = 'close')
plt.legend()
plt.title('Stock Analysis Simon Property Group for retail stock')
plt.show()

In [ ]:
EXPE_final = final_data[final_data.stock_name == 'EXPE']
EXPE_final1 = EXPE_final.groupby('timestamp')['open','high','low','close'].mean().reset_index().merge(EXPE_final.groupby('timestamp')['death_cases','confirmed_cases'].sum().reset_index(),
                                                                                                   on = 'timestamp')

EXPE_final1['death_cases'] = (EXPE_final1.death_cases - EXPE_final1.death_cases.mean())/EXPE_final1.death_cases.std()
EXPE_final1['confirmed_cases'] = (EXPE_final1.confirmed_cases - EXPE_final1.confirmed_cases.mean())/EXPE_final1.confirmed_cases.std()
EXPE_final1['high'] = (EXPE_final1.high - EXPE_final1.high.mean())/EXPE_final1.high.std()
EXPE_final1['low'] = (EXPE_final1.low - EXPE_final1.low.mean())/EXPE_final1.low.std()
EXPE_final1['open'] = (EXPE_final1.open - EXPE_final1.open.mean())/EXPE_final1.open.std()
EXPE_final1['close'] = (EXPE_final1.close - EXPE_final1.close.mean())/EXPE_final1.close.std()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(EXPE_final1.timestamp, EXPE_final1.death_cases, label = 'death_cases')
plt.plot(EXPE_final1.timestamp, EXPE_final1.confirmed_cases, label = 'confirmed_cases')
plt.plot(EXPE_final1.timestamp, EXPE_final1.high, label = 'high')
plt.plot(EXPE_final1.timestamp, EXPE_final1.low, label = 'low')
plt.plot(EXPE_final1.timestamp, EXPE_final1.open, label = 'open')
plt.plot(EXPE_final1.timestamp, EXPE_final1.close, label = 'close')
plt.legend()
plt.title('Stock Analysis Expedia for travel stock')
plt.show()

In [ ]:
GOLD_final = final_data[final_data.stock_name == 'GOLD']
GOLD_final1 = GOLD_final.groupby('timestamp')['open','high','low','close'].mean().reset_index().merge(GOLD_final.groupby('timestamp')['death_cases','confirmed_cases'].sum().reset_index(),
                                                                                                   on = 'timestamp')

GOLD_final1['death_cases'] = (GOLD_final1.death_cases - GOLD_final1.death_cases.mean())/GOLD_final1.death_cases.std()
GOLD_final1['confirmed_cases'] = (GOLD_final1.confirmed_cases - GOLD_final1.confirmed_cases.mean())/GOLD_final1.confirmed_cases.std()
GOLD_final1['high'] = (GOLD_final1.high - GOLD_final1.high.mean())/GOLD_final1.high.std()
GOLD_final1['low'] = (GOLD_final1.low - GOLD_final1.low.mean())/GOLD_final1.low.std()
GOLD_final1['open'] = (GOLD_final1.open - GOLD_final1.open.mean())/GOLD_final1.open.std()
GOLD_final1['close'] = (GOLD_final1.close - GOLD_final1.close.mean())/GOLD_final1.close.std()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(GOLD_final1.timestamp, GOLD_final1.death_cases, label = 'death_cases')
plt.plot(GOLD_final1.timestamp, GOLD_final1.confirmed_cases, label = 'confirmed_cases')
plt.plot(GOLD_final1.timestamp, GOLD_final1.high, label = 'high')
plt.plot(GOLD_final1.timestamp, GOLD_final1.low, label = 'low')
plt.plot(GOLD_final1.timestamp, GOLD_final1.open, label = 'open')
plt.plot(GOLD_final1.timestamp, GOLD_final1.close, label = 'close')
plt.legend()
plt.title('Stock Analysis GOLD for material stock')
plt.show()